In [2]:
library(spectralGraphTopology)
library(assertthat)
options(warn = -1)



Attaching package: 'spectralGraphTopology'


The following object is masked from 'package:stats':

    D


Warning message:
"package 'assertthat' was built under R version 4.5.1"


In [91]:
# Load library for multivariate normal distribution
library(MASS)

# Set parameters
set.seed(123)       # For reproducibility
n <- 1000            # Number of time points
p <- 4              # Number of variables

mu <- runif(p, min = -1, max = 1) # random means between -1 and 1

A <- matrix(rnorm(p^2), nrow = p)
SigmaMat <- crossprod(A)  # t(A) %*% A, guarantees positive definite

# Covariance matrix
Sigma <- matrix(SigmaMat, nrow = p, byrow = TRUE)

# Generate multivariate normal data
data <- mvrnorm(n = n, mu = mu, Sigma = Sigma)

# Convert to time series object
ts_data <- ts(data, start = c(2020, 1), frequency = 12)

# View first few rows
head(ts_data)

Jan 2020,0.6052612,1.9354184,0.3949486,-1.0786222
Feb 2020,-3.4464044,1.6086381,-2.7426028,0.2372721
Mar 2020,-2.4463587,-0.3511798,-0.4878841,3.5643112
Apr 2020,-0.6468191,-0.7836053,-0.9615872,1.1145096
May 2020,-4.4504753,-0.5788189,-3.2866664,1.3011199
Jun 2020,-2.9747249,1.0322598,-1.5268351,1.7996007


In [92]:


X <- scale(as.matrix(ts_data))
  
# number of nodes
p <- ncol(X)
  
# number of observations
T_n <- nrow(X)

w0 <- "naive"
Sinv <- MASS::ginv(cor(X))
w <- spectralGraphTopology:::w_init(w0, Sinv)

w0 <- Linv(Sinv)
w0[w0 < 0] <- 0


print(Sinv)

print("")
print(w)
#print(w0)

          [,1]      [,2]       [,3]       [,4]
[1,]  31.08849 11.010645 -20.763591  25.469668
[2,]  11.01065  5.269683  -7.224546   9.717319
[3,] -20.76359 -7.224546  14.880659 -16.971989
[4,]  25.46967  9.717319 -16.971989  22.221386
[1] ""


[1]  0.000000 20.763591  0.000000  7.224546  0.000000 16.971989


In [94]:
A0 <- A(w0)
A0 <- A0 / rowSums(A0)
print(" ")
print(A0)


[1] " "
          [,1]      [,2] [,3]      [,4]
[1,] 0.0000000 0.0000000    1 0.0000000
[2,] 0.0000000 0.0000000    1 0.0000000
[3,] 0.4618223 0.1606879    0 0.3774898
[4,] 0.0000000 0.0000000    1 0.0000000


In [95]:
Ainv <- function(M) {
  N <- ncol(M)
  w <- c()
  for (i in 1:(N-1)) {
    for (j in (i+1):N) {
      w <- c(w, M[i, j])
    }
  }
  return(w)
}

w <- Ainv(A0)
print(" ")
print("Custom Function ")
print(w)

w <- spectralGraphTopology:::Ainv(A0)
print(" ")
print("Spectral Graph Topology ")
print(w)

[1] " "
[1] "Custom Function "
[1] 0.0000000 1.0000000 0.0000000 1.0000000 0.0000000 0.3774898
[1] " "
[1] "Spectral Graph Topology "
[1] 0.0000000 1.0000000 0.0000000 1.0000000 0.0000000 0.3774898


In [96]:
Linv <- function(M) {
  N <- ncol(M)
  w <- c()
  for (i in 1:(N-1)) {
    for (j in (i+1):N) {
      w <- c(w, -M[i, j])  # negative of off-diagonal
    }
  }
  return(w)
}


w <- Linv(Sinv)
print(" ")
print("Custom Function ")
print(w)

w <- spectralGraphTopology:::Linv(Sinv)
print(" ")
print("Spectral Graph Topology ")
print(w)

[1] " "
[1] "Custom Function "
[1] -11.010645  20.763591 -25.469668   7.224546  -9.717319  16.971989
[1] " "
[1] "Spectral Graph Topology "
[1] -11.010645  20.763591 -25.469668   7.224546  -9.717319  16.971989


In [99]:
# Build Laplacian-like matrix L from edge-weight vector w
L <- function(w) {
  k <- length(w)
  n <- as.integer(0.5 * (1 + sqrt(1 + 8 * k)))  # solve n(n-1)/2 = k
  Lw <- matrix(0, n, n)

  pos <- 1
  # fill the UPPER triangle row-wise with -w
  for (i in 1:(n - 1)) {
    j <- n - i                       # how many entries to place in row i
    Lw[i, (i + 1):n] <- -w[pos:(pos + j - 1)]
    pos <- pos + j
  }

  # set diagonal to minus column sums of (Lw + t(Lw))
  LwColSum <- colSums(Lw + t(Lw))
  diag(Lw) <- diag(Lw) - LwColSum

  # return symmetric matrix using the upper triangle as source
  Lw[lower.tri(Lw)] <- t(Lw)[lower.tri(Lw)]
  return(Lw)
}


Lw <- spectralGraphTopology:::L(w)
print(" ")
print("Spectral Graph Topology ")
print(Lw)

[1] " "
[1] "Spectral Graph Topology "
          [,1]       [,2]       [,3]       [,4]
[1,] -15.71672  11.010645 -20.763591  25.469668
[2,]  11.01065 -13.503418  -7.224546   9.717319
[3,] -20.76359  -7.224546  44.960126 -16.971989
[4,]  25.46967   9.717319 -16.971989 -18.214997


In [59]:
k = 3
Lw <- L(w)
Aw <- A(w)
U <- eigen(Lw, symmetric = TRUE)$vectors[, (p - k + 1):p]
print(U)

U <- eigen(Lw, symmetric = TRUE)
print(U)

           [,1]          [,2]       [,3]
[1,]  0.4082483  7.071068e-01 -0.5773503
[2,]  0.4082483 -7.071068e-01 -0.5773503
[3,] -0.8164966 -8.881784e-16 -0.5773503
eigen() decomposition
$values
[1] 3.000000e+00 1.000000e+00 2.664535e-15

$vectors
           [,1]          [,2]       [,3]
[1,]  0.4082483  7.071068e-01 -0.5773503
[2,]  0.4082483 -7.071068e-01 -0.5773503
[3,] -0.8164966 -8.881784e-16 -0.5773503



In [ ]:
a <- rep(1, p*(p-1)/2)
a

[1] 1 1 1

: 